In [ ]:
import pandas as pd
import os
import torch
import torch.nn as nn

import dataloader
import utils

from model import generate_model
from optimizer import Adam,SGD
from train_wrapper import train_epoch
import datetime

In [ ]:
config = utils.load_config()

In [ ]:
df_dataset = pd.read_csv(config['DATASET_PATH'])
df_dataset = df_dataset.dropna().reset_index(drop=True)

from sklearn.model_selection import train_test_split
X = df_dataset.drop(labels='label',axis=1)
Y = df_dataset['label']

X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,shuffle=True,stratify=None,random_state=1004)

In [ ]:
batch_size = 32
num_workers = 16
learning_rate = 0.001

model_name = config['model']['model_name']
model_depth = config['model']['model_depth']
config['save_datetime'] = datetime.datetime.now().strftime("%Y%m%d_%H%M")

In [ ]:
traindata=dataloader.CTDataset(X_train,y_train)
valdata=dataloader.CTDataset(X_test,y_test)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(traindata , batch_size=batch_size, shuffle=True, sampler = None, num_workers=num_workers,pin_memory = True)
val_dataloader = DataLoader(valdata , batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory = True)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model, _ = generate_model(model_name=model_name,model_depth = model_depth,n_classes=2,resnet_shortcut='B',add_last_fc_num = 0)
model.to(device)

if torch.cuda.is_available():
    model = nn.DataParallel(model, device_ids = [0,1,2,3])
    model.to(device)
    
optimizer = Adam(model, learning_rate = learning_rate)
criterion_clf = nn.CrossEntropyLoss().to(device)

In [ ]:
utils.save_messgage(config
                    , model_name=model_name, model_depth = model_depth, n_classes=2, resnet_shortcut='B'
                    , optimizer = 'Adam', lr=learning_rate, criterion_clf='CrossEntropyLoss'
                    , message='비고')


train_epoch(device,train_dataloader,val_dataloader,val_dataloader,model,criterion_clf,optimizer,config,epoch = 100,num_classes=2)